# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [43]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV



# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [29]:
df_cls = pd.read_csv("dataset_clustering.csv")

In [30]:
df_cls.sample(7)

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,TransactionType,Channel,CustomerOccupation,Cluster_Final
1045,230.08000,79.0,124.0,1.0,4466.10,Debit,Online,Retired,0
1156,913.49125,64.0,75.0,2.0,3173.17,Debit,ATM,Retired,0
2012,434.67000,27.0,124.0,1.0,1298.13,Credit,Online,Student,1
2152,202.49000,48.0,69.0,1.0,7447.10,Credit,ATM,Engineer,0
548,124.59000,27.0,171.0,1.0,129.29,Debit,Branch,Student,1
1666,506.03000,21.0,21.0,1.0,1123.89,Debit,Branch,Student,1
1668,350.96000,63.0,167.0,1.0,5463.94,Debit,ATM,Retired,0


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [31]:
df_cls['Cluster_Final'].value_counts()

Cluster_Final
0    1859
1     626
2      27
Name: count, dtype: int64

In [ ]:
# Memisahkan fitur dan target
X = df_cls.drop(columns='Cluster_Final')
y = df_cls['Cluster_Final']

# Mengidentifikasi fitur kategorikal dan numerikal
cat_cols = ['TransactionType', 'Channel', 'CustomerOccupation']
num_cols = X.columns.difference(cat_cols)

# One-hot encoding untuk fitur kategorikal
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_cat_encoded = pd.DataFrame(
    ohe.fit_transform(X[cat_cols]),
    columns=ohe.get_feature_names_out(cat_cols),
    index=X.index
)

# Menggabungkan kembali fitur numerik dan hasil encoding
X_full = pd.concat([
    X[num_cols].reset_index(drop=True),
    X_cat_encoded.reset_index(drop=True)
], axis=1)

# Split data latih (train) dan uji (test)
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y, test_size=0.3, random_state=42, 
)

# Normalisasi hanya pada fitur numerik
scaler_cls = StandardScaler()
X_train[num_cols] = scaler_cls.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler_cls.transform(X_test[num_cols])

In [38]:
print("Train:")
print(y_train.value_counts())
print("\nTest:")
print(y_test.value_counts())

Train:
Cluster_Final
0    1290
1     447
2      21
Name: count, dtype: int64

Test:
Cluster_Final
0    569
1    179
2      6
Name: count, dtype: int64


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [39]:
# Menerapkan 2 algoritma klasifikasi sebagai perbandingan model
# Model 1: Random Forest Classifier
cls_rf = RandomForestClassifier(n_estimators=100, random_state=42)
cls_rf.fit(X_train, y_train)

# Model 2: Logistic Regression
cls_logreg = LogisticRegression(max_iter=500, random_state=42)
cls_logreg.fit(X_train, y_train)


LogisticRegression(max_iter=500, random_state=42)

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

*1. Random Forest Classifier*
- Random Forest dipilih sebagai model pertama karena kemampuannya yang kuat dalam menangani data dengan kombinasi fitur numerik dan kategorikal.
- Algoritma ini bekerja dengan membangun banyak pohon keputusan (decision trees) dan menggabungkan hasilnya untuk meningkatkan akurasi prediksi.
- Sangat andal dalam mengatasi overfitting dan cukup tangguh terhadap noise maupun outlier pada data.
- Mengingat dataset ini terdiri dari fitur transaksi yang beragam, Random Forest cocok karena mampu menangani kompleksitas dan interaksi antar fitur dengan baik.
- Selain itu, model ini juga menyediakan feature importance, yang bermanfaat untuk mengetahui fitur-fitur mana yang paling berkontribusi dalam membedakan antar klaster.

*2. Logistic Regression*
- Logistic Regression digunakan sebagai model pembanding karena sifatnya yang sederhana dan mudah diinterpretasikan.
- Cocok untuk klasifikasi multi-kelas seperti prediksi hasil segmentasi KMeans yang menghasilkan lebih dari dua klaster.
- Algoritma ini efektif untuk data yang sudah distandarisasi, seperti yang telah dilakukan pada pipeline preprocessing.
- Logistic Regression juga berguna untuk mengevaluasi apakah terdapat hubungan linier antara fitur input (misalnya usia atau jumlah login) dengan kemungkinan keanggotaan pada masing-masing klaster.
- Meskipun sederhana, Logistic Regression tetap memberikan baseline performa yang cukup baik dan mudah dianalisis.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [ ]:
# Prediksi model dengan data uji
# Prediksi menggunakan Random Forest (Model 1)
y_pred_rf_train = cls_rf.predict(X_train)
y_pred_rf_test = cls_rf.predict(X_test)

# Prediksi menggunakan Logistic Regression (Model 2)
y_pred_lr_train = cls_logreg.predict(X_train)
y_pred_lr_test = cls_logreg.predict(X_test)

In [90]:
# Evaluasi Akurasi Model
accuracy_rf = accuracy_score(y_test, y_pred_rf_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr_test)
print("\n=== Accuracy Score ===")
print(f"Random Forest Testing Accuracy: {accuracy_rf:.4f}")
print(f"Logistic Regression Testing Accuracy: {accuracy_lr:.4f}")

# Evaluasi F1-Score (weighted average)
print("\n=== F1-Score (Training & Testing) ===")
print(f"Random Forest - Train: {f1_score(y_train, y_pred_rf_train, average='weighted'):.4f}")
print(f"Random Forest - Test : {f1_score(y_test, y_pred_rf_test, average='weighted'):.4f}")
print(f"Logistic Reg - Train : {f1_score(y_train, y_pred_lr_train, average='weighted'):.4f}")
print(f"Logistic Reg - Test  : {f1_score(y_test, y_pred_lr_test, average='weighted'):.4f}")

# Classification Report
print("\n=== Classification Report ===")
print("\nClassification Report - Random Forest")
print(classification_report(y_test, y_pred_rf_test))

print("\nClassification Report - Logistic Regression")
print(classification_report(y_test, y_pred_lr_test))

# Confusion Matrix
print("\n=== Confusion Matrix ===")
print("\nConfusion Matrix - Random Forest")
print(confusion_matrix(y_test, y_pred_rf_test))

print("\nConfusion Matrix - Logistic Regression")
print(confusion_matrix(y_test, y_pred_lr_test))



=== Accuracy Score ===
Random Forest Testing Accuracy: 0.9881
Logistic Regression Testing Accuracy: 0.9907

=== F1-Score (Training & Testing) ===
Random Forest - Train: 1.0000
Random Forest - Test : 0.9881
Logistic Reg - Train : 0.9989
Logistic Reg - Test  : 0.9907

=== Classification Report ===

Classification Report - Random Forest
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       569
           1       0.98      0.97      0.97       179
           2       1.00      1.00      1.00         6

    accuracy                           0.99       754
   macro avg       0.99      0.99      0.99       754
weighted avg       0.99      0.99      0.99       754


Classification Report - Logistic Regression
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       569
           1       0.98      0.98      0.98       179
           2       1.00      1.00      1.00         6

    accuracy       

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

*1. Evaluasi Model: Random Forest Classifier*
- Akurasi (Test Set): 98.81%
- F1-Score Training: 100%
- F1-Score Testing: 98.81%
- Precision, Recall, F1-Score:
    - Kelas 0: precision dan recall sangat tinggi (0.99)
    - Kelas 1: performa sangat baik, recall sedikit turun (0.97)
    - Kelas 2: diprediksi sempurna (1.00 untuk semua metrik)

- Confusion Matrix:
    - Kelas 0: 565 benar, 4 salah diklasifikasikan sebagai kelas 1
    - Kelas 1: 174 benar, 5 salah diklasifikasikan sebagai kelas 0
    - Kelas 2: 6 data diprediksi dengan benar (akurasi 100%)

- Insight:
    - Model ini sangat akurat namun menunjukkan tanda overfitting (F1 training sempurna).
    - Masih sangat kompeten di data uji, dengan kesalahan kecil pada kelas mayoritas (0 dan 1).

*2. Evaluasi Model: Logistic Regression*
- Akurasi (Test Set): 99.07%
- F1-Score Training: 99.89%
- F1-Score Testing: 99.07%
- Precision, Recall, F1-Score:
    - Kelas 0 & 2: precision dan recall sempurna atau nyaris sempurna
    - Kelas 1: performa tetap tinggi (98%)

- Confusion Matrix:
    - Kelas 0: 566 benar, 3 salah diklasifikasikan sebagai kelas 1
    - Kelas 1: 175 benar, 4 salah diklasifikasikan sebagai kelas 0
    - Kelas 2: 6 data diprediksi benar seluruhnya

- Insight:
    - Lebih stabil dan tidak overfitting dibanding Random Forest.
    - Mampu menangani kelas minoritas (kelas 2) dengan akurasi tinggi.
    - Konsisten di data latih dan uji.

*Kesimpulan:*
- Kedua model bekerja sangat baik, dengan akurasi di atas 98%.
- Logistic Regression sedikit lebih unggul secara keseluruhan dari sisi akurasi dan stabilitas.
- Random Forest cenderung overfit, tetapi tetap akurat dan powerful.
- Untuk model yang mudah diinterpretasi dan tahan overfitting, Logistic Regression lebih direkomendasikan.
- Random Forest tetap layak digunakan, terutama bila dilakukan tuning lanjutan atau ensemble.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [91]:
# Hyperparameter tuning untuk Random Forest
# Menyusun grid hyperparameter yang akan diuji
rf_params_adjusted = {
    'n_estimators': [100, 200, 300],                 # jumlah pohon
    'max_depth': [3, 6, 8, 10],                      # kedalaman pohon
    'min_samples_split': [5, 7, 10],                 # minimal sampel untuk split
    'min_samples_leaf': [2, 4, 7, 8],                # minimal sampel per daun
    'max_features': ['sqrt', 'log2']                 # jumlah fitur saat split
}

# Inisialisasi model Random Forest dasar
rf_base = RandomForestClassifier(random_state=42)

# Setup GridSearchCV
rf_grid = GridSearchCV(
    estimator=rf_base,
    param_grid=rf_params_adjusted,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

# Melatih  GridSearch menggunakan data training
rf_grid.fit(X_train, y_train)

# Menampilkan parameter terbaik dan skor validasi tertinggi
print("Best Hyperparameters:", rf_grid.best_params_)
print("Best Cross-Validated Accuracy:", rf_grid.best_score_)



Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Best Hyperparameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best Cross-Validated Accuracy: 0.9863523051023051


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [92]:
# Mengambil parameter terbaik dari hasil GridSearch
best_params = rf_grid.best_params_

# Inisialisasi ulang model Random Forest dengan parameter terbaik
rf_optimized = RandomForestClassifier(**best_params, random_state=42)
rf_optimized.fit(X_train, y_train)

# Prediksi pada data train dan test
y_train_pred_rf = rf_optimized.predict(X_train)
y_test_pred_rf = rf_optimized.predict(X_test)

In [93]:
# Evaluasi Akurasi
train_acc = accuracy_score(y_train, y_train_pred_rf)
test_acc = accuracy_score(y_test, y_test_pred_rf)

print("\n=== Accuracy Score ===")
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy : {test_acc:.4f}")

# Evaluasi F1 Score
f1_train = f1_score(y_train, y_train_pred_rf, average='weighted')
f1_test = f1_score(y_test, y_test_pred_rf, average='weighted')

print("\n=== F1 Score ===")
print(f"Train F1-Score: {f1_train:.4f}")
print(f"Test F1-Score : {f1_test:.4f}")

# Classification Report
print("\n=== Classification Report ===")
print("\nClassification Report (Test Data)")
print(classification_report(y_test, y_test_pred_rf))

# Confusion Matrix
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_test_pred_rf))


=== Accuracy Score ===
Train Accuracy: 0.9983
Test Accuracy : 0.9867

=== F1 Score ===
Train F1-Score: 0.9982
Test F1-Score : 0.9867

=== Classification Report ===

Classification Report (Test Data)
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       569
           1       0.97      0.97      0.97       179
           2       1.00      1.00      1.00         6

    accuracy                           0.99       754
   macro avg       0.99      0.99      0.99       754
weighted avg       0.99      0.99      0.99       754


=== Confusion Matrix ===
[[564   5   0]
 [  5 174   0]
 [  0   0   6]]


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.



## 1. Perbandingan Model (Sebelum dan Sesudah Tuning)
### a. Perbandingan Model Random Forest (Sebelum dan Sesudah Tuning)

**Tabel Perbandingan:**

| Metrik             | Sebelum Tuning (RF) | Sesudah Tuning (RF)        |
|--------------------|---------------------|-----------------------------|
| Train Accuracy     | 1.0000 (100%) ✅     | 0.9983 (99.83%)             |
| Test Accuracy      | 0.9881 (98.81%)     | 0.9867 (98.67%)             |
| F1-Score Train     | 1.0000 (100%) ✅     | 0.9982 (99.82%)             |
| F1-Score Test      | 0.9881 (98.81%)     | 0.9867 (98.67%)             |


**Kesimpulan:**
- Setelah tuning, **overfitting berhasil dikurangi**. Hal ini ditandai dengan skor training yang turun sedikit (tidak lagi 100%).
- Performa testing tetap sangat baik (di atas 98%), meskipun ada sedikit penurunan F1-Score.
- Model menjadi lebih **generalizable**, tidak hanya menghafal data latih.

---
### b. Hasil Model Logistic Regression  
**Metrik Evaluasi:**

| Class | Precision | Recall | F1-Score | Support |
|-------|-----------|--------|----------|---------|
| 0     | 0.99      | 0.99   | 0.99     | 569     |
| 1     | 0.98      | 0.98   | 0.98     | 179     |
| 2     | 1.00      | 1.00   | 1.00     | 6       |
|       |           |        |          |         |
| **Overall Accuracy** |       |        | **0.9907** | **754** |

- Logistic Regression menunjukkan kinerja sangat baik dan konsisten.
- Akurasi tinggi pada data uji mengindikasikan model mampu menggeneralisasi dengan baik, tanpa overfitting.
- F1-Score untuk semua kelas berada di atas 0.98, menunjukkan kualitas prediksi yang merata.
- Kelas 1 memiliki skor sedikit lebih rendah dibanding kelas lain, namun masih tergolong sangat baik — hal ini bisa dipengaruhi oleh ukuran support yang lebih kecil dibanding kelas mayoritas.
- Secara keseluruhan, model ini tepat digunakan untuk segmentasi klasifikasi awal karena stabil dan cukup akurat di semua kelas.


---

## 2. Identifikasi Kelemahan Model
### a. Precision atau Recall
#### Random Forest:
- **Kelas 1** tetap menjadi kelas yang paling menantang, dengan F1-Score 0.97. Meskipun tergolong tinggi, nilainya sedikit lebih rendah dibanding kelas lainnya.
- **Kelas 2** memiliki metrik sempurna (precision dan recall = 1.00), tetapi ini perlu diwaspadai karena jumlah datanya hanya 6. Dengan data yang sangat kecil, performa ini mudah berubah jika ada sedikit perubahan pada data.
- **Sebelum tuning**, model menunjukkan gejala overfitting, ditandai dengan F1-Score sempurna di data latih.
- **Setelah tuning**, overfitting berhasil ditekan dan model menjadi lebih seimbang. Namun, hal ini disertai sedikit penurunan akurasi di data uji — sebuah trade-off yang umum terjadi saat meningkatkan generalisasi.


#### Logistic Regression:
- **Kelas 1** kembali menjadi yang paling rendah performanya, dengan F1-Score 0.98. Meski tetap sangat baik, perbedaan ini menunjukkan kelas tersebut lebih sulit diprediksi secara konsisten.
- **Kelas 2** diprediksi sempurna, tetapi seperti halnya pada Random Forest, jumlah datanya sangat sedikit sehingga metrik ini tidak bisa dijadikan acuan yang terlalu kuat.
- Secara keseluruhan, model ini tidak memiliki kelemahan mencolok, karena precision dan recall di semua kelas ≥ 0.98. Namun, pemantauan terhadap kelas minoritas tetap disarankan seiring bertambahnya data.


---

### b. Overfitting / Underfitting

- **Sebelum tuning**: Model Random Forest mengalami overfitting, karena skor training = 100%.
- **Sesudah tuning**: Overfitting berkurang. Gap antara train dan test lebih kecil.
- Tidak ada indikasi underfitting karena semua skor sangat tinggi baik di training maupun testing.


---
## 3. Rekomendasi Tindakan Lanjutan
a. Perluasan dan Pemerataan Data
- Disarankan untuk menambah data pada kelas yang jumlahnya sedikit (terutama kelas 2) supaya model bisa belajar lebih banyak pola.
- Jika penambahan data sulit dilakukan, bisa coba pendekatan seperti *oversampling* (misalnya SMOTE) atau undersampling agar distribusi label jadi lebih seimbang.

b. Eksplorasi Model Lain
- Uji model dengan daya adaptif lebih tinggi terhadap data imbalance, seperti:
  - Boosting-based models (XGBoost, LightGBM, CatBoost)
  - SVM dengan penyesuaian bobot antar kelas
- Tujuannya untuk membandingkan apakah model lain bisa memberikan hasil yang lebih baik atau lebih stabil dari Random Forest.

c. Peningkatan Evaluasi
- Untuk menguji kestabilan performa model, bisa pakai *cross-validation 10-fold* agar model divalidasi di berbagai kombinasi data.
- Selain itu, metode seperti *stratified shuffle split* juga bisa dipakai agar pembagian data tetap proporsional tiap kelasnya.

d. Optimasi Fitur
- Lakukan analisis terhadap kontribusi setiap fitur, misalnya pakai visualisasi seperti *SHAP values* atau *permutation importance*.
- Coba buat fitur turunan baru dari fitur yang sudah ada, atau gunakan teknik seperti interaction features.
- Jika ada fitur yang terlalu mirip, bisa dicek korelasinya dan disederhanakan supaya model tidak bingung saat belajar.

---

### Kesimpulan Akhir

- **Logistic Regression** terbukti menjadi model dengan performa paling stabil dan akurat dalam pengujian ini. Model ini menunjukkan nilai akurasi dan f1-score yang tinggi, serta tidak mengalami overfitting, baik pada data latih maupun data uji.
- **Random Forest** , meskipun pada awalnya mengalami overfitting, menunjukkan peningkatan setelah dilakukan *tuning hyperparameter*. Nilai akurasi pada data latih menurun secara wajar, yang mengindikasikan model menjadi lebih general dan tidak hanya menyesuaikan diri terhadap data pelatihan.
- Secara keseluruhan, kedua model menghasilkan performa yang sangat baik, dengan **akurasi dan f1-score di atas 98%**. Namun, perhatian khusus masih perlu diberikan terhadap **kelas minoritas (kelas 2)** karena jumlah datanya sangat terbatas, sehingga berisiko menurunkan kestabilan prediksi apabila terdapat perubahan distribusi data.
- Langkah selanjutnya sebaiknya fokus pada:
  - Perluasan Data Kelas Minor
  - Uji Model Lain
  - Tingkatkan Validasi
  - Optimalisasi Fitur
